#  ブラックボックス最適化(BBO)ベンチマーク

## 手法
### 遺伝的アルゴリズム

### 進化戦略
https://www.jstage.jst.go.jp/article/sicejl/54/8/54_567/_pdf
http://www.matsumoto.nuem.nagoya-u.ac.jp/jascome/denshi-journal/20/No.08-201219.pdf
https://arxiv.org/abs/1604.00772
https://horomary.hatenablog.com/entry/2021/01/23/013508
https://math-note.com/multivariate-normal-distribution/

In [ ]:
import numpy as np
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
import os
import datetime

from abc import ABCMeta,abstractmethod
from typing import List, Dict,Tuple

from bbo import GeneticAlgorithm,BBO,CMA_ES,TPE,RandomSearch,GridSearch,NSGA,NelderMead

In [ ]:
# https://qiita.com/nabenabe0928/items/08ed6495853c3dd08f1e
class Model(metaclass=ABCMeta):
    @abstractmethod
    def evaluation(self,params:List[any])->float:
        pass

    @abstractmethod
    def get_params_range(self)->List[Tuple[float]]:
        pass

In [ ]:
class QuadraticFunction(Model):
    def __init__(self,dim=10,scale = 10):
        self._dim = dim
        self._range = (-1*scale,1*scale)

    def evaluation(self,params):
        if len(params) != self._dim:
            print('Error: \'params\' must be {} dimentions'.format(self._dim), file=sys.stderr)
            sys.exit(1)
        else:
            params = np.array(params)
            eval = np.sum(params*params)
            return eval

    def get_params_range(self):
        range_s = [self._range for i in range(self._dim)]
        return range_s

In [ ]:
class AckleyFunction(Model):
    def __init__(self,dim=10,scale = 30):
        self._dim = dim
        self._range = (-1*scale,1*scale)
        self.boundaries = np.array(self._range)

    def evaluation(self, params):
        x = np.array(params)
        t1 = 20
        t2 = - 20 * np.exp(- 0.2 * np.sqrt(1.0 / len(x) * np.sum(x ** 2)))
        t3 = np.e
        t4 = - np.exp(1.0 / len(x) * np.sum(np.cos(2 * np.pi * x)))
        return t1 + t2 + t3 + t4

    def get_params_range(self):
        range_s = [self._range for i in range(self._dim)]
        return range_s

In [ ]:
class RosenbrockFunction(Model):
    def __init__(self,dim=10,scale = 5):
        self._dim = dim
        self._range = (-1*scale,1*scale)
        self.boundaries = np.array(self._range)

    def evaluation(self, params):
        x = np.array(params)
        val = 0
        for i in range(0, len(x) - 1):
            t1 = 100 * (x[i + 1] - x[i] ** 2) ** 2
            t2 = (x[i] - 1) ** 2
            val += t1 + t2
        return val
        
    def get_params_range(self):
        range_s = [self._range for i in range(self._dim)]
        return range_s

In [ ]:
class StyblinskiTangFunction(Model):
    def __init__(self,dim=10,scale = 5):
        self._dim = dim
        self._range = (-1*scale,1*scale)
        self.boundaries = np.array(self._range)

    def evaluation(self, params):
        x = np.array(params)
        t1 = np.sum(x ** 4)
        t2 = - 16 * np.sum(x ** 2)
        t3 = 5 * np.sum(x)
        return 0.5 * (t1 + t2 + t3) + 39.166165*self._dim

    def get_params_range(self):
        range_s = [self._range for i in range(self._dim)]
        return range_s

In [ ]:
class GriewankFunction(Model):
    def __init__(self,dim=10,scale = 5):
        self._dim = dim
        self._range = (-1*scale,1*scale)
        self.boundaries = np.array(self._range)

    def evaluation(self, params):
        x = np.array(params)
        w = np.array([1.0 / np.sqrt(i + 1) for i in range(len(x))])
        t1 = 1
        t2 = 1.0 / 4000.0 * np.sum(x ** 2)
        t3 = - np.prod(np.cos(x * w))
        return t1 + t2 + t3

    def get_params_range(self):
        range_s = [self._range for i in range(self._dim)]
        return range_s

In [ ]:
class SchwefelFunction(Model):
    def __init__(self,dim=10,scale = 500):
        self._dim = dim
        self._range = (-1*scale,1*scale)
        self.boundaries = np.array(self._range)

    def evaluation(self, params):
        x = np.array(params)
        return - np.sum(x * np.sin( np.sqrt( np.abs(x) ) ) ) + 418.9829*self._dim

    def get_params_range(self):
        range_s = [self._range for i in range(self._dim)]
        return range_s

In [ ]:
class XinSheYangFunction(Model):
    def __init__(self,dim=10,scale = 6):
        self._dim = dim
        self._range = (-1*scale,1*scale)
        self.boundaries = np.array(self._range)

    def evaluation(self, params):
        x = np.array(params)
        t1 = np.sum( np.abs(x) )
        e1 = - np.sum( np.sin(x ** 2) )
        t2 = np.exp(e1)
        return t1 * t2

    def get_params_range(self):
        range_s = [self._range for i in range(self._dim)]
        return range_s

In [ ]:
class BenchMarker():
    def __init__(self,max_iter=100):
        self._bbo_table = {
            'CMA-ES':CMA_ES(),
            'TPE':TPE(),
            'RandomSearch':RandomSearch(),
            'GridSearch':GridSearch(),
            'GA(optuna)':NSGA(),
            'Nelder-Mead':NelderMead(),
            'GA(hand made)':GeneticAlgorithm(),            
        }
        
        self._max_iter = max_iter
        self._result_s = dict()

    def benchmark(self,model,is_plot=False):
        result = dict()
        for key,value in self._bbo_table.items():
            print("{} start!".format(key))
            fval,params = value.optimization(model,self._max_iter)
            result.update({key:(fval,params)})
            fval_s,params_s = value.get_history()
            self._result_s.update({key:(fval_s,params_s)})

        for key,value in result.items():
            print("{} achieved {}.".format(key,value[0]))

        if is_plot:
            self.plot_history()

    def plot_history(self,is_log = False,is_save=True):
        plt.figure()
        trial = [i for i in range(self._max_iter)]
        for key,value in self._result_s.items():
            plt.plot(trial,value[0],label=key)
        plt.legend()
        if is_log:
            plt.yscale('log')
        if is_save:
            os.makedirs('fig',exist_ok=True)
            now = datetime.datetime.now()
            d = now.strftime('%Y%m%d%H%M%S')
            plt.savefig('fig/'+d+'.png', dpi=300)
        plt.show()      



In [ ]:
dim = 100
model_s = [QuadraticFunction(dim=dim),
            StyblinskiTangFunction(dim=dim),
            GriewankFunction(dim=dim),
            AckleyFunction(dim=dim),
            RosenbrockFunction(dim=dim),
            SchwefelFunction(dim=dim),
            XinSheYangFunction(dim=dim),
        ]
for model in model_s:
    bm = BenchMarker(max_iter=1000)
    bm.benchmark(
                    model = model,
                    is_plot=True
                )
    bm.plot_history(is_log=True)